In [ ]:
from bs4 import BeautifulSoup
import urllib3
from urllib.request import urlopen
import smtplib
from flask import Flask, request, Response
import re
import time 
from textblob import TextBlob

In [ ]:
def make_soup(url):
    html = urlopen(url)
    return BeautifulSoup(html, 'html.parser')

In [ ]:
url = "https://www.pc-kombo.com/us/components/cpus"
soup = make_soup(url)


lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
final = []
ibi = 0
for i in lista:
    
    i = i.find('a')
    if(i != 'None' and i != -1):
        ibi += 1
        nombre = i.find("h5", {'class': 'name'})
        subtitle = i.find("div", {'class': 'subtitle'}).find_all('span')
        
        print("Nombre: " + str(nombre.get_text(strip=True)))
        #print("Cores: "+ str(cores.get_text(strip=True)))
        print("Socket: ", end = "")
        nombre,socket,reloj,ok,cores,threads
        for j in subtitle:
            j = str(j.get_text(strip=True))
            if(len(j) > 2):
                print(j)
        print("----------------------------------------------------------------")
    
print(ibi)   

In [ ]:
url = "https://www.pc-kombo.com/us/components/gpus"
soup = make_soup(url)


lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
final = []
ibi = 0
for i in lista:
    
    i = i.find('a')

    if(i != 'None' and i != -1):
        ibi += 1
        nombre = i.find("h5", {'class': 'name'})
        subtitle = i.find("div", {'class': 'subtitle'}).find_all('span')
        print(i)
        print("Link: " + str(i.find('href')))
        print("Nombre: " + str(nombre.get_text(strip=True)))
        #print("Cores: "+ str(cores.get_text(strip=True)))
        print("Socket: ", end = "")
        for j in subtitle:
            j = str(j.get_text(strip=True))
            if(len(j) > 2):
                print(j)
        print("----------------------------------------------------------------")
    
print(ibi)   

In [22]:
def scrapping_motherboards():

    url = "https://www.pc-kombo.com/us/components/gpus"
    soup = make_soup(url)

    lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
    final = []
    ibi = 0
    for i in lista:

        i = (i.find('a', href=True)['href']).split(" ")[0]

        if(i != 'None' and i != -1):
            ibi+=1
            print(i)

            url_2 = i
            soup_2 = make_soup(url_2)

            price = soup_2.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'})
            
            if price == None:
                print("PRECIO:", price)
            else:
                print("PRECIO:", price.text)
            list_a = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
            list_b = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')

            for i in range(len(list_a)):
                print(list_b[i].text, " ", list_a[i].text)
            print("----------------------------------------------------------------")

    print(ibi)
    
scrapping_motherboards()

https://www.pc-kombo.com/us/product/gpu/4895106290662_Sapphire
PRECIO: 279.99
Producer   SAPPHIRE
MPN   11310-01-20G
EAN   4895106290662
UPC   
Year   
Length   193 mm
Slots   2.5
8-pin connectors   1
6-pin connectors   0
HDMI   1
DisplayPort   3
DVI   0
VGA   0
Boost Clock   2044 MHz
Vram   8 GB
Memory Clock   0 MHz
Sync   
TDP   132 W
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/gpu/4719331307844_Aorus
PRECIO: 629.99
Producer   Gigabyte
MPN   GV-N3070AORUS M-8GD
EAN   4719331307844
UPC   
Year   
Length   290 mm
Slots   3.0
8-pin connectors   2
6-pin connectors   0
HDMI   3
DisplayPort   3
DVI   0
VGA   0
Boost Clock   1845 MHz
Vram   8 GB
Memory Clock   14000 MHz
Sync   G-Sync
TDP   220 W
Product Page   
----------------------------------------------------------------
https://www.pc-kombo.com/us/product/gpu/4711081635321_ASUS


KeyboardInterrupt: 

In [ ]:
url = "https://www.pc-kombo.com/us/product/motherboard/4719072733667_MSI%20B550-A%20Pro"
soup = make_soup(url)

price = soup.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'}).text
print("PRECIO:", price)
list_a = soup.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
list_b = soup.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')

for i in range(len(list_a)):
    print(list_b[i].text, " ", list_a[i].text)
    

In [40]:
def scrapping_all_components(code):

    url = "https://www.pc-kombo.com/us/components/" + code
    soup = make_soup(url)
    columns_scrapping = ['Producer', 'Length', 'Slots', '8-pin connectors', '6-pin connectors', 'Vram',
                         'Boost Clock', 'Memory Clock']
    lista = [m for m in soup.find("body").find("ol", {'id': 'hardware'}).find_all('li')]
    final = []
    ibi = 0
    
    for i in lista:

        i = (i.find('a', href=True)['href']).split(" ")[0]

        if(i != 'None' and i != -1):
            ibi+=1
            print(i)

            url_2 = i
            soup_2 = make_soup(url_2)
            
            name = soup_2.find("body").find("h1", {"itemprop":"name"}).text
            
            rating = str(soup_2.find("body").find('p', {'itemprop' : 'aggregateRating'}).text).replace(" ", "")
            rating = rating.replace('\n', '').replace('\r', '')
            rating = rating[-2:-9:-1]
            price = soup_2.find("body").find("section", {'class': 'column col-12 col-md-12'}).find('span', {'itemprop':'price'})
            list_a = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dd')
            list_b = soup_2.find("body").find('div', {'class': 'container grid-xl'}).find('div', {'class': 'columns'}).find_all('dt')
            
            #ELEMNTOS PARA LA TABLA: NAME, PRODUCER, MODEL, LENGTH, SLOTS, 8_PIN, 6_PIN,
            # VRAM, TDP, BOOST_CLOCK, MEMORY_CLOCK, PERFORMANCE_LEVEL, PRICE
            if price == None:
                print(name + "," + str(price) + "," + rating[::-1] + ",",  end = "")
            else:
                print(name + "," + str(price.txt) + "," + rating[::-1] + ",", end = "")

            for i in range(len(list_a)):
                
                if list_b[i].text in columns_scrapping:
                    print(list_a[i].text, end = "")
                    print(",", end="")
                   
            print("")

    print(ibi)
    
scrapping_all_components("gpus")

https://www.pc-kombo.com/us/product/gpu/4895106290662_Sapphire
Sapphire Pulse Radeon RX 6600 Gaming,None,4.06/10,SAPPHIRE,193 mm,2.5,1,0,2044 MHz,8 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331307844_Aorus
Aorus RTX 3070 Master,None,6.52/10,Gigabyte,290 mm,3.0,2,0,1845 MHz,8 GB,14000 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081635321_ASUS
ASUS GeForce RTX 3050 Dual,None,3.73/10,ASUS,200 mm,2.0,1,0,1852 MHz,8 GB,4 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081645221_ASUS
ASUS GeForce RTX 3050 Phoenix,None,3.73/10,ASUS,177 mm,2.55,1,0,1807 MHz,8 GB,4 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081645443_ASUS
ASUS GeForce RTX 3050 ROG Strix,None,3.73/10,ASUS,300 mm,2.7,1,0,1890 MHz,8 GB,4 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081310020_ASUS
ASUS GeForce RTX 3060 Dual LHR,None,4.79/10,ASUS,200 mm,2.5,1,0,1807 MHz,12 GB,15000 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081127178_ASUS
ASUS GeForce RTX 3060 Dual OC,None,4.79/10,ASUS,200 mm,2.5,1,

ASUS Radeon RX 6700 XT ROG Strix O12G,None,t6.5/10,ASUS,322 mm,3.0,2,0,2622 MHz,12 GB,8000 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081119289_ASUS
ASUS Radeon RX 6700 XT TUF O12G,None,t6.5/10,ASUS,322 mm,3.0,1,1,2622 MHz,12 GB,8000 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081731740_ASUS
ASUS Radeon RX 6750 XT Dual,None,6.54/10,ASUS,295 mm,2.7,2,0, MHz,12 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4711081730934_ASUS
ASUS Radeon RX 6750 XT ROG Strix,None,6.54/10,ASUS,322 mm,2.9,2,0, MHz,12 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4718017977760_ASUS
ASUS Radeon RX 6800,None,6.55/10,ASUS,267 mm,2.0,2,0,2105 MHz,16 GB,8000 MHz,
https://www.pc-kombo.com/us/product/gpu/4718017988018_ASUS
ASUS Radeon RX 6800 ROG Strix,None,6.55/10,ASUS,320 mm,3.0,2,0,2190 MHz,16 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4718017990004_ASUS
ASUS Radeon RX 6800 TUF,None,6.55/10,ASUS,320 mm,3.0,2,0,2190 MHz,16 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4718017990202_A

Gigabyte Aorus GeForce RTX 3050 Elite,None,3.73/10,Gigabyte,296 mm,2.5,1,0,1860 MHz,8 GB,4 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331309329_Gigabyte
Gigabyte Aorus GeForce RTX 3060 Elite 12G LHR,None,4.79/10,Gigabyte,296 mm,2.75,1,1,1867 MHz,12 GB,15000 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331309541_Gigabyte
Gigabyte Aorus GeForce RTX 3060 Ti Elite 8G LHR,None,6.37/10,Gigabyte,296 mm,2.7,1,1,1750 MHz,8 GB,14000 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331307967_Gigabyte
Gigabyte Aorus GeForce RTX 3060 Ti Master,None,6.37/10,Gigabyte,290 mm,3.0,1,1, MHz,8 GB,14000 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331309534_Gigabyte
Gigabyte Aorus GeForce RTX 3070 Master 8G LHR,None,6.52/10,Gigabyte,290 mm,3.0,2,0, MHz,8 GB,14000 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331309657_Gigabyte
Gigabyte Aorus GeForce RTX 3080 Master 10G LHR,None,6.57/10,Gigabyte,319 mm,3.5,3,0,1845 MHz,10 GB,19000 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331308865_G

Gigabyte GeForce RTX 3090 Gaming OC,None,6.62/10,Gigabyte,320 mm,2.75,2,0,1755 MHz,24 GB,19500 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331310776_Gigabyte
Gigabyte GeForce RTX 3090 Ti Gaming,None,6.63/10,Gigabyte,331 mm,3.5,0,0,1860 MHz,24 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331310653_Gigabyte
Gigabyte GeForce RTX 3090 Ti Gaming OC,None,6.63/10,Gigabyte,331 mm,3.5,0,0,1905 MHz,24 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331310554_Gigabyte
Gigabyte Radeon RX 6500 XT Eagle,None,t3.6/10,Gigabyte,192 mm,2.0,0,1,2815 MHz,4 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331310561_Gigabyte
Gigabyte Radeon RX 6500 XT Gaming OC,None,t3.6/10,Gigabyte,282 mm,2.0,0,1,2825 MHz,4 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331309923_Gigabyte
Gigabyte Radeon RX 6600 Eagle,None,4.06/10,Gigabyte,282 mm,2.0,1,0,2491 MHz,8 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4719331309749_Gigabyte
Gigabyte Radeon RX 6600 XT Eagle,None,5.39/10,Gigabyte,

INNO3D GeForce RTX 3090 iChill Frostbite,None,6.62/10,INNO3D,226 mm,2.0,2,0,1755 MHz,24 GB,19500 MHz,
https://www.pc-kombo.com/us/product/gpu/4895223101520_INNO3D
INNO3D GeForce RTX 3090 iChill X3,None,6.62/10,INNO3D,300 mm,3.0,2,0,1755 MHz,24 GB,19500 MHz,
https://www.pc-kombo.com/us/product/gpu/0835168001534_INNO3D
INNO3D GeForce RTX 3090 iChill X4,None,6.62/10,INNO3D,300 mm,3.0,2,0,1755 MHz,24 GB,19500 MHz,
https://www.pc-kombo.com/us/product/gpu/0835168002661_INNO3D
INNO3D GeForce RTX 3090 Ti X3 OC,None,6.63/10,INNO3D,330 mm,3.0,0,0,1890 MHz,24 GB,0 MHz,
https://www.pc-kombo.com/us/product/gpu/4895223102275_INNO3D
INNO3D GeForce RTX 3090 X3,None,6.62/10,INNO3D,300 mm,2.0,3,0,1695 MHz,24 GB,19500 MHz,
https://www.pc-kombo.com/us/product/gpu/0835168001602_INNO3D
INNO3D RTX 3070 iChill X3,None,6.52/10,INNO3D,300 mm,3.0,2,0,1785 MHz,8 GB,14000 MHz,
https://www.pc-kombo.com/us/product/gpu/0835168001596_INNO3D
INNO3D RTX 3070 iChill X4,None,6.52/10,INNO3D,300 mm,3.0,2,0,1785 MHz,8 GB,140

KeyboardInterrupt: 

In [ ]:
#ELEMNTOS PARA LA TABLA: NAME, PRODUCER, MODEL, LENGTH, SLOTS, 8_PIN, 6_PIN, VRAM, TDP, BOOST_CLOCK, MEMORY_CLOCK, PERFORMANCE_LEVEL, PRICE